In [ ]:
import ipywidgets as widgets
import time
from IPython.display import display, clear_output, HTML, Javascript
from tapestrysdk import fetch_group_list, fetch_group_document, generate_report

tapestry_ids=8

def fetch_groups(tapestry_id):
    try:
        response = fetch_group_list(tapestry_id)
        if response.get("success") and "body" in response:
            return response["body"]
        else:
            return []
    except Exception:
        return []
    
def fetch_documents(group_id):
    try:
        tapestry_id = tapestry_ids
        response = fetch_group_document(tapestry_id, group_id)
        if response.get("success") and "body" in response:
            return response["body"]
        else:
            return []
    except Exception:
        return []

def main():
    groups_cache = {}
    documents_cache = {}

    # initial default prompt
    current_prompt = {"text": """ """}

    # --- Fetch groups from SDK ---
    groups = fetch_groups(tapestry_ids)
    if not groups:
        groups = []
    for g in groups:
        groups_cache[g["id"]] = g

    # --- Global CSS ---
    styles = """
    <style>
    #rendered_cells {
        background-color: #F7F7FF !important;
    }
    .main_content {
        display: flex;
        flex-direction: column;
        gap: 10px;
    }
    .section-box {
        display: flex;
        flex-direction: column;
    }
    .choose_doc_card__wrapper {
        display: flex;
        justify-content: center;
    }
    .choose_card {
        display: flex;
        flex-direction: column;
        gap: 15px;
        background-color: #7F7F80;
        border-radius: 20px;
        width: 100%;
        padding: 12px;
    }
    .groupDropdown select {
        border-radius: 5px;
        cursor: pointer;
    }
    .documents_container {
        position: relative;
        background-color: white;
        border-radius: 20px;
        height: 250px;
        max-width: 700px;
        overflow: auto;
        padding: 8px;
    }
    .documents_container div {
        width: auto;
        min-height: 30px;
        display: flex;
        align-items: center;
        margin-block: 3px;
        padding: 4px;
        background-color: #F2F2F2;
        border-radius: 5px;
    }
    .documents_container label {
        display: flex;
        align-items: center;
    }
    .documents_container .widget-checkbox input[type="checkbox"] {
        margin-right: 30px;
    }
    .analyse_button {
        display: flex;
        align-items: center;
        justify-content: center;
        border-radius: 5px;
        padding: 5px 0;
    }
    .navbar {
        display: flex;
        justify-content: flex-start;
        padding: 10px;
    }
    .navbar button {
        display: flex;
        align-items: center;
        justify-content: center;
        background: #444;
        color: white;
        padding: 10px 16px;
        margin-right: 8px;
        border: none;
        border-radius: 5px;
        cursor: pointer;
    }
    .navbar button:hover {
        background: #666;
    }
    .prompt_page {
        width: auto;
        display: flex;
        flex-direction: column;
        gap: 10px;
        background-color: #7F7F80;
        border-radius: 20px;
        padding: 12px;
    }
    .prompt_input, .prompt_input textarea  {
        width: auto;
        border-radius: 20px;
    }
    .prompt_input textarea  {
        padding: 10px;
        width: auto;
        height: 350px;
    }

    /* Full screen overlay */
    .fullscreen-loader {
        position: fixed;  /* cover whole screen */
        top: 0;
        left: 0;
        inset: 0;
        width: 100%;
        height: 100vh;
        display: flex;
        justify-content: center;
        align-items: center;
        background: rgba(255,255,255,0.6);
        backdrop-filter: blur(4px); /* blur background */
        z-index: 2000;
    }

    /* Spinner */
    .fullscreen-loader .spinner {
        border: 6px solid #f3f3f3;
        border-top: 6px solid #3498db;
        border-radius: 50%;
        width: 48px;
        height: 48px;
        animation: spin 1s linear infinite;
    }

    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }


    .result_doc_row {
        display: flex;
        align-items: center;
        justify-content: space-between;
        padding: 4px 8px;
        margin: 5px 5px;
        border-radius: 5px;
        background-color: #F2F2F2;
    }

    .results_box {
        margin-top: 10px;
        border-radius: 20px;
        background-color: #7F7F80;
        display: flex;
        flex-direction: column;
        gap: 5px
    }

    .download_btn {
        background-color: black;
        color: white;
        padding: 4px 11px;
        border: none;
        border-radius: 20px;
        font-weight: 600;
    }

    .result_wrapper {
        padding: 10px;
    }

    .change_prompt_button {
        border-radius: 5px;
    }
    </style>
    """
    display(HTML(styles))

    # --- Loader widget (CSS spinner) ---
    fullscreen_loader = widgets.HTML(
        """<div id="widget-loader" class="fullscreen-loader"><div class="spinner"></div></div>"""
    )
    fullscreen_loader.layout.display = "none"

    # --- Navbar ---
    analyse_btn = widgets.Button(description="Analyse", layout=widgets.Layout(width="120px"))
    prompt_btn = widgets.Button(description="Prompt", layout=widgets.Layout(width="120px"))
    navbar = widgets.HBox([analyse_btn, prompt_btn])
    navbar.add_class("navbar")

    # --- Analyse Page ---
    documents_container = widgets.VBox([])
    documents_container.add_class("documents_container")
    
    group_dropdown = widgets.Dropdown(
        options=[(g["name"], g["id"]) for g in groups] or [("No groups available", None)],
        layout=widgets.Layout(width="300px")
    )
    group_dropdown.add_class("groupDropdown")

    choose_group_section = widgets.VBox([
        widgets.HTML("<h3>Select group</h3>"),
        group_dropdown,
    ])
    choose_group_section.add_class("section-box")

    choose_documents_section = widgets.VBox([
        widgets.HTML("<h3>Select documents</h3>"),
        documents_container,
    ])
    choose_documents_section.add_class("section-box")

    choose_doc_card = widgets.VBox([
        choose_group_section,
        choose_documents_section
    ])
    choose_doc_card.add_class("choose_doc_card")
    choose_doc_card__wrapper = widgets.VBox([choose_doc_card])
    choose_doc_card__wrapper.add_class("choose_doc_card__wrapper")

    choose_param_section = widgets.VBox([
        widgets.Checkbox(value=False, description="Consider chat", indent=False)
    ])

    analyse_button = widgets.Button(
        description="Run Analysis", 
        button_style="primary", 
        layout=widgets.Layout(width="200px")
    )
    analyse_button.add_class("analyse_button")

    choose_card = widgets.VBox([
        choose_doc_card__wrapper,
        choose_param_section,
        analyse_button
    ])
    choose_card.add_class("choose_card")
    results_box = widgets.VBox([])
    results_box.add_class("results_box")

    analyse_page = widgets.VBox([choose_card, results_box], layout=widgets.Layout(padding="15px"))

    # --- Prompt Page ---
    text_input = widgets.Textarea(
        value=current_prompt["text"],
        description="Prompt:", 
        layout=widgets.Layout(height='350px')
    )
    text_input.style.description_width = "0px"
    text_input.add_class("prompt_input")

    change_prompt_button = widgets.Button(
        description="Save prompt", 
        button_style="primary", 
        layout=widgets.Layout(width="200px")
    )
    change_prompt_button.add_class("change_prompt_button")

    def on_save_prompt(b):
        show_loader()
        current_prompt["text"] = text_input.value
        time.sleep(1.1) 
        with main_content:
            clear_output(wait=True)
            display(prompt_page)
        hide_loader()
        # toast removed

    change_prompt_button.on_click(on_save_prompt)

    prompt_page = widgets.VBox([
        widgets.HTML("<h3>Edit prompt</h3>"),
        text_input,
        change_prompt_button
    ], layout=widgets.Layout(padding="15px"))
    prompt_page.add_class("prompt_page")

    # --- Handler ---
    def show_loader():
        fullscreen_loader.layout.display = "flex"

    def hide_loader():
        fullscreen_loader.layout.display = "none"

    def on_group_change(change):
        if change["name"] == "value" and change["new"] is not None:
            group_id = change["new"]

            show_loader()
            try:
                docs = fetch_documents(group_id)
                documents_cache[group_id] = docs
                if docs:
                    checkboxes = [widgets.Checkbox(
                        value=False,
                        description=d.get("name", f"Doc {d['id']}"),
                        indent=False
                    ) for d in docs]
                    documents_container.children = checkboxes
                else:
                    documents_container.children = [widgets.Label("No documents found")]
            except Exception as e:
                documents_container.children = [widgets.Label(f"⚠️ Error: {e}")]
            finally:
                hide_loader()

    group_dropdown.observe(on_group_change, names="value")

    # --- Main Content Area ---
    main_content = widgets.Output()
    main_content.add_class("main_content")

    def show_analyse(_=None):
        with main_content:
            clear_output()
            display(analyse_page)

    def show_prompt(_=None):
        with main_content:
            clear_output()
            display(prompt_page)

    analyse_btn.on_click(show_analyse)
    prompt_btn.on_click(show_prompt)

    def on_analyse_click(b):
        results_box.children = []  # reset

         # collect selected doc IDs
        selected_docs = [
            d["id"] for i, d in enumerate(documents_cache.get(group_dropdown.value, []))
            if isinstance(documents_container.children[i], widgets.Checkbox) 
            and documents_container.children[i].value
        ]
        if not selected_docs:
            results_box.children = [widgets.Label("⚠️ No documents selected!")]
            return
         # prepare params
        tapestry_id = tapestry_ids
        group_id = group_dropdown.value
        is_chat = choose_param_section.children[0].value
        user_prompt = text_input.value

        show_loader()
        try:
            # call SDK
            response = generate_report(
                tapestry_id=tapestry_id,
                group_id=group_id,
                document_ids=selected_docs,
                is_chat=is_chat,
                user_prompt=user_prompt
            )

            # render response
            if response.get("success") and "body" in response:
                docs = response["body"]  # dict {title: url}
                
                res_header = widgets.HTML("<h3>Reports</h3>")

                rows = []

                for title, url in docs.items():
                    download_link = widgets.HTML(
                        f'<a href="{url}" target="_blank" download style="text-decoration:none;">'
                        f'<button class="download_btn">Download</button>'
                        '</a>'
                    )
                    box = widgets.HBox([
                        widgets.HTML(f"<span>📄 {title}</span>"),
                        download_link
                    ])
                    box.add_class("result_doc_row")
                    rows.append(box)
                
                result_wrapper = widgets.VBox([
                    res_header,
                    *rows
                ])
                result_wrapper.add_class("result_wrapper")
                results_box.children =  [result_wrapper]
            else:
                results_box.children = [widgets.Label("⚠️ Analysis failed or empty response.")]
        except Exception as e:
            results_box.children = [widgets.Label(f"⚠️ Error: {e}")]
        finally:
            hide_loader()

    analyse_button.on_click(on_analyse_click)

    # --- Final App Layout ---
    app = widgets.VBox([navbar, main_content, fullscreen_loader])
    with main_content:
        show_analyse()
    display(app)

    # ✅ Trigger document load for initially selected group
    if group_dropdown.value:
        on_group_change({"name": "value", "new": group_dropdown.value})

# Run app once
main()